In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options # 브라우저 옵션 설정 (ex. headless 모드)
from selenium.webdriver.chrome.service import Service # 브라우저 실행 서비스 설정 모듈 
from selenium.webdriver.common.by import By # WebDriver에서 요소를 찾는 데 사용되는 방법을 지정
from selenium.webdriver.common.keys import Keys # 특수 키(예: Enter, Shift 등)를 보냄
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException #  NoSuchElement(특정 요소가 없는 경우),UnexpectedAlert(브라우저 경고 ex. 삭제된 페이지 입니다.) 


from os import rename, listdir
from datetime import datetime 
import time
import csv
import re
import pandas as pd
from tqdm import tqdm

/Users/user/first_pytorch/lib/python3.7/site-packages/urllib3/__init__.py:38: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  exceptions.NotOpenSSLWarning,


In [2]:
chrome_driver = '/Users/user/Desktop/haeyoon_private/cafeteria/chromedriver-mac-x64/chromedriver'

In [3]:
# ChromeDriver 서비스 객체 생성
service = Service(chrome_driver)

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
#options.add_argument('--headless')  # headless 모드 사용 (브라우져창 열지 않음.)

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://auth.band.us/login_page") # 해당 링크 오픈 
driver.implicitly_wait(3) # 요소 노출 까지 3초 대기


In [4]:
fin_df = pd.DataFrame()
post_no_list = range(926901171,926901865)

In [4]:
fin_df = pd.read_csv('/Users/user/Desktop/haeyoon_private/cafeteria/data/raw_data.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
post_no_list = range(926900590,926901170)

In [5]:
for post_no in tqdm(post_no_list) :
    driver.get("https://band.us/band/47113445/post/"+str(post_no))
    time.sleep(1)
    try:
        # 포스트
        post_body = ""
        post_body = driver.find_element(By.CSS_SELECTOR, 'div.txtBody').text # 포스트 내용
        if 'course' in post_body.lower(): # course라는 단어가 들어있는 글이 메뉴 관련 글임. 
            post_writer = driver.find_element(By.CSS_SELECTOR, 'a.text').text # 포스트 작성자
            post_write_time= driver.find_element(By.CSS_SELECTOR, 'time.time').text # 작성 시간 (포스트, 댓글) 

            df = pd.DataFrame(
                        data = {
                            'post_no' : [post_no],
                            'type' : ['body'],
                            'writer' : [post_writer],
                            'body': [post_body],
                            'time' : [post_write_time]
                        }
                    )
            fin_df = pd.concat([
                                fin_df,
                                df
                            ])
            # 댓글
            item_wrap_elements = driver.find_elements(By.CSS_SELECTOR, 'div.itemWrap')
            for item in item_wrap_elements:
                try: 
                    comment_body = item.find_element(By.CSS_SELECTOR, 'p.txt._commentContent').text
                except (NoSuchElementException, UnexpectedAlertPresentException):
                    comment_body = ''
                    
                df = pd.DataFrame(
                            data = {
                                'post_no' : [post_no],
                                'type' : 'comment',
                                'writer' : [item.find_element(By.CSS_SELECTOR, 'strong.name').text],
                                'body': [comment_body],
                                'time' : [item.find_element(By.CSS_SELECTOR, 'time.time').text]
                            }
                        )

                fin_df = pd.concat([
                                    fin_df,
                                    df
                                ])

            fin_df.to_csv('/Users/user/Desktop/haeyoon_private/cafeteria/data/raw_data.csv', sep=',' ,na_rep='NaN',index = False)

        
    except (NoSuchElementException,UnexpectedAlertPresentException):
            # 에러 발생 시 무시하고 다음 루프로 넘어감
            continue

100%|██████████| 580/580 [23:26<00:00,  2.42s/it]
